<a href="https://colab.research.google.com/github/Solrak97/clasificador_de_sentimientos/blob/main/second_model_speech_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Busqueda de entradas para el modelo base propuesto por Dias Issa et al.
El modelo base cuenta con una entrada de 193 nodos, por lo que es necesario que cada entrada de datos tenga exactamente 193 variables, eso a partir de las ya conocidas caracteristicas 

* Chromagram
* Contrast
* Mel coeficent
* Mel frequency
* Tonnetz

### Descarga de los datos desde Kaggle hasta el notebook.

In [4]:
%%capture
! pip install kaggle wavio pydub keras-metrics
! rm -rf *
! mkdir ~/.kaggle
! curl https://raw.githubusercontent.com/Solrak97/clasificador_de_sentimientos/main/kaggle.json > kaggle.json
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download uwrfkaggler/ravdess-emotional-speech-audio
! unzip ravdess-emotional-speech-audio.zip

import soundfile
import numpy as np
import librosa
import glob
import os
import pickle
from pydub import AudioSegment
import seaborn as sns
from matplotlib import pyplot as plt

### Algoritmo de extracción de datos

In [ ]:
def extract_feature(file_name, **kwargs):
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz") 

    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate

        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
      
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=128).T, axis=0)
            result = np.hstack((result, mfccs))
           
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
           
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
           
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
           
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
           
    return result
  

In [5]:
# we allow only these emotions ( feel free to tune this on your need )
AVAILABLE_EMOTIONS = {
    "neutral",
    "calm",
    "happy",
    "sad",
    "angry",
    "fearful",
    "disgust",
    "surprised"
}

'''
Modality (01 = full-AV, 02 = video-only, 03 = audio-only).
Vocal channel (01 = speech, 02 = song).
Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.
Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").
Repetition (01 = 1st repetition, 02 = 2nd repetition).
Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).
'''

INT_2_MODALITY = {
    '01' : 'full-AV',
    '02' : 'video-only',
    '03' : 'audio-only'
}

INT_2_VOCAL = {
    '01' : 'speech',
    '02' : 'song'
}

INT_2_EMOTION = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

INT_2_INTENSITY = {
    '01' : 'normal', 
    '02' : 'strong'
}

INT_2_STATEMENT = {
    '01' : '"Kids are talking by the door"',
    '02' : '"Dogs are sitting by the door"'
}

INT_2_REPETITION = {
    '01' : '1st repetition', 
    '02' : '2nd repetition'
}

In [5]:
import pandas as pd


def load_data():

    _modality = []
    _vocal_channel = []
    _emotions = []
    _intensity = []
    _statement = []
    _repetition = []
    _actor = []
    _features = []

    X, y = [], []

    for file in glob.glob("Actor_*/*.wav"):
            
      file_name = os.path.basename(file)
      
      # El audio viene en estereo para algunas partes
      # Así que se pasa a mono
      sound = AudioSegment.from_wav(file)
      sound = sound.set_channels(1)
      sound.export(file, format="wav")
  
      name_split = file_name.split("-")
      emotion = INT_2_EMOTION[name_split[2]]

      # Limitación de emociones
      if emotion not in AVAILABLE_EMOTIONS:
        continue
      
      _modality.append(INT_2_MODALITY[name_split[0]])
      _vocal_channel.append(INT_2_VOCAL[name_split[1]])
      _intensity.append(INT_2_INTENSITY[name_split[3]])
      _statement.append(INT_2_STATEMENT[name_split[4]])
      _repetition.append(INT_2_REPETITION[name_split[5]])
      _actor.append(name_split[6])
      _emotions.append(emotion)

      # Extracción de los datos graciosos
      features = extract_feature(file, mfcc=True, 
                                 chroma=True, mel=True, 
                                 contrast=True, tonnetz=True)
      
      _features.append(features)


    data = {
      'Modality' :  _modality,
      'Vocal Channel' : _vocal_channel,
      'Emotion' : _emotions,
      'Intensity' : _intensity,
      'Statement' : _statement,
      'Repetition' : _repetition,
      'Actor_ID' : _actor,
      'Features' : _features
    } 

    df = pd.DataFrame(data)

    return df

In [ ]:
data = load_data()

## Plan para el modelo 02:
1. Separar el training (80%) del test (20%)
1. Agrandar el Sample
  1. Modificar con distintas velocidades
    1. velocidades: { 0.81%, 1.00%, 1.23% }
    1. leer un audio, modificar su velocidad y reproducirlo
    1. de todo el dataset tomar un 80% de entrenamiento y 20 de testeo
    1. cada audio en el dataset de entrenamiento debe contar con una versión en cada una de las velocidades
    1. se pasa de tener 425 a 1275
  1. Agregar ruido para duplicar el sample
    1. para cada uno de los audios en las distintas velocidades, se agrega un ruido de 25%
    1. se pasa de tener 1275 a tener 2550
1. Replicar los pasos del grafo en la figura 4 en la fabricación del modelo
1. Construir la matriz de confusión

In [6]:
from pydub import AudioSegment


def cambio_velocidad(sound, speed=1.0):

    sound_with_altered_frame_rate = sound._spawn(sound.raw_data, overrides={
         "frame_rate": int(sound.frame_rate * speed)
      })

    return sound_with_altered_frame_rate.set_frame_rate(sound.frame_rate)


#slow_sound = cambio_velocidad(my_sound, 0.81)
#fast_sound = cambio_velocidad(my_sound, 1.23)

In [7]:
from numpy.core.fromnumeric import var
import math

def agregar_ruido(audio_name, seconds_duration):
  # Se agrega ruido a la 1ra cuarta parte del audio
  signal, sr = librosa.load(audio_name, sr = 16000, duration=seconds_duration*0.25)
  noise = np.random.normal(0, 0.001, signal.shape[0])
  noise_sound = signal + noise
  
  soundfile.write('noisePart.wav', noise_sound, 16000)
  noisePart = AudioSegment.from_wav('/content/noisePart.wav')

  # 75% del audio original
  originalPart = AudioSegment.from_wav(audio_name)
  originalPart = originalPart[seconds_duration * 1000 * 0.25:seconds_duration * 1000]

  result = noisePart + originalPart

  return result

### Función que aumenta el tamaño del dataset haciendo uso de las funsiones cambio_velocidad() y agregar_ruido()

In [19]:
def incrementa_dataset(file_name):

  sound_array = []

  for file in glob.glob(file_name):
          
    file_name = os.path.basename(file)
    full_name = os.path.abspath(file)
    
    # El audio viene en estereo para algunas partes así que se pasa a mono
    sound = AudioSegment.from_wav(file)
    sound = sound.set_channels(1)
    sound.export(file, format="wav")

    name_split = file_name.split("-")
    emotion = INT_2_EMOTION[name_split[2]]

    # Agregamos ruido al audio original
    sound_duration = sound.duration_seconds
    noise_sound = agregar_ruido(full_name, sound_duration)

    # agregamos los audios en velocidad 1.00%
    sound_array.append((sound, emotion))
    sound_array.append((noise_sound, emotion))

    # agregamos versión de audios a 0.81%
    slow_sound = cambio_velocidad(sound, 0.81)
    noise_slow_sound = cambio_velocidad(noise_sound, 0.81)
    sound_array.append((slow_sound, emotion))
    sound_array.append((noise_slow_sound, emotion))

    # agregamos versión de audios a 1.23%
    fast_sound = cambio_velocidad(sound, 1.23)
    noise_fast_sound = cambio_velocidad(noise_sound, 1.23)
    sound_array.append((fast_sound, emotion))
    sound_array.append((noise_fast_sound, emotion))

  return sound_array

In [20]:
my_sound_array = incrementa_dataset("Actor_*/*.wav")

### Pruebas con casos particulares

In [41]:
entry_number = int(input('Entry number:\t'))
print('\nThe subject is', my_sound_array[entry_number][1], '\n')
my_sound = my_sound_array[entry_number][0]
my_sound

Entry number:	53

The subject is happy 



In [42]:
slow_sound = cambio_velocidad(my_sound, 0.81)
slow_sound

In [43]:
fast_sound = cambio_velocidad(my_sound, 1.23)
fast_sound

## Prueba de ruido, 1ro sin ruido, 2do con ruido en la 1ra cuarta parte del audio

In [32]:
sound_without_noise = AudioSegment.from_wav('/content/Actor_18/03-01-02-01-02-02-18.wav')
sound_without_noise

In [ ]:
sound = AudioSegment.from_wav('/content/Actor_18/03-01-02-01-02-02-18.wav')
sound_with_noise = agregar_ruido('/content/Actor_18/03-01-02-01-02-02-18.wav', sound.duration_seconds)
sound_with_noise

# Posible generación del modelo

## Imports para generación del modelo

In [ ]:
import keras
from keras import Sequential
from keras.layers import Dropout, MaxPool1D, Flatten, Dense, ReLU, Input, BatchNormalization, Softmax
from keras.layers.convolutional import Conv1D

## Ruta segundo modelo

In [ ]:
from keras.layers.advanced_activations import Softmax
def build_baseline():
  model = Sequential()

  # Input
  model.add(Input(shape=(193, 1)))

  # 1era y 2da convolusion
  model.add(Conv1D(strides=1, filters=255, kernel_size=5))
  model.add(BatchNormalization())
  model.add(ReLU())

  model.add(Conv1D(strides=1, filters=128, kernel_size=5))
  model.add(ReLU())

  # Dropout
  model.add(Dropout(rate=0.1))
  model.add(BatchNormalization())

  # Max pooling
  model.add(MaxPool1D(pool_size=8))

  # Convolusion intermedia
  model.add(Conv1D(strides=1, filters=128, kernel_size=5))
  model.add(BatchNormalization())
  model.add(ReLU())

  # Dropout
  model.add(Dropout(rate=0.2))

  # Flatten
  model.add(Flatten())

  # Capa densa
  model.add(Dense(units=8))
  model.add(BatchNormalization())

  # Softmax
  model.add(Softmax())

  return model